In [1]:
import pandas as pd 
import numpy as np
import regex as re

#### Sentiment Analysis

In [2]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
df_sent = pd.read_csv("Rumours_NER.csv", index_col = [0] )
df = df_sent

In [4]:
#Words which affect in the wrong way, remove them from the rumours text
df_sent["Rumour"] = df_sent["Rumour"].str.replace("attacking", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("attack", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("Rob", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("Angers", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("lower", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("contenders", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("battle", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("serious", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("cancel", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("struggling", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("accidentally", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("must", "")
df_sent["Rumour"] = df_sent["Rumour"].str.replace("demanding", "")


In [5]:
#nltk.download("vader_lexicon")

In [6]:
analyzer = SentimentIntensityAnalyzer()

In [7]:
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    compound_score = scores["compound"]

    if compound_score >= 0.05:
        return "Positive"
    elif compound_score > -0.05 and compound_score < 0.05:
        return "Neutral"
    else:
        return "Negative"

In [8]:
#This allows to iterate through the clubs despite it being converted to string rather than list when opening
from ast import literal_eval

In [9]:
#Splitting the rumour where a club name is recognised, then taking the 3 following words and getting a sentiment. 
#If for one of the clubs is negative, we say the rumour has a negative score
score = [] 
for i in range(len(df_sent)) : #len(df
    temp = []
    for club in literal_eval(df_sent.iloc[i,3]) :
        my_regex = r'\b' + club + r'\b(?:\s*\S+\s*?){3}'
        text = re.findall(my_regex, df.iloc[i,0])
        temp.append(get_sentiment("".join(text)))
    if "Negative" in temp : 
        score.append("Negative")
    else : 
         score.append("Positive")    

In [10]:
#Code to print which lines are negative

#for i in range(len(df_sent)) : #len(df
#    temp = []
#    for club in literal_eval(df_sent.iloc[i,3]) :
#        my_regex = r'\b' + club + r'\b(?:\s*\S+\s*?){3}'
#        text = re.findall(my_regex, df.iloc[i,0])
#        if get_sentiment("".join(text)) == "Negative" : 
#            print(text)
#            print(df_sent.iloc[i,0])
#

In [11]:
df["Score"] = score

In [12]:
df.to_csv('Rumours_Sent.csv', encoding='utf-8')

In [13]:
len(list(df_sent.loc[df_sent["Score"] == "Negative", "Rumour"]))

71